<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/ff_analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers==4.15.0 --quiet
!pip install -U sentence-transformers==2.2.0 --quiet
!pip install -U transformers[sentencepiece] --quiet
!pip install datasets --quiet

     |████████████████████████████████| 3.4 MB 10.2 MB/s 
     |████████████████████████████████| 3.3 MB 48.6 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 880 kB 62.2 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 17.6 MB/s 
     |████████████████████████████████| 346 kB 9.0 MB/s 
     |████████████████████████████████| 212 kB 43.8 MB/s 
     |████████████████████████████████| 1.1 MB 47.0 MB/s 
     |████████████████████████████████| 140 kB 55.7 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 127 kB 57.1 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 271 kB 59.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the pac

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd

In [ ]:
k = pd.read_csv("ff_train_new.csv")


In [ ]:
import torch
  
print(f"Is CUDA supported by this system? \
      {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:\
      {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:\
      {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system?       True
CUDA version: 11.3
ID of current CUDA device:      0
Name of current CUDA device:      Tesla T4


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer
import pandas as pd
from datasets import Dataset, DatasetDict


#raw_datasets = load_dataset("glue", "mrpc")
raw_datasets = pd.read_csv("ff_train_new.csv")
#raw_datasets = raw_datasets.drop('Unnamed: 0', axis=1)
raw_datasets = raw_datasets.rename(columns={'Summary': 'text', 'Unnamed: 0': 'ID'})

#raw_datasets = raw_datasets.drop(['Unnamed: 0.1', 'work_id'], axis = 1)
raw_datasets = Dataset.from_dict(raw_datasets)

# 90% train, 10% test + validation
ttv_ds = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = ttv_ds['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': ttv_ds['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

#checkpoint = "bert-base-uncased"
checkpoint = "zdreiosis/bert-finetuned-sem_eval-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


#def tokenize_function(example):
#    return tokenizer(example["text"], truncation=True)


#tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading file https://huggingface.co/zdreiosis/bert-finetuned-sem_eval-english/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/90d5bec27d0cb8fae7d40cbf20597cf26b34b7eb85fdb6203f05819a5c3b2b5e.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/zdreiosis/bert-finetuned-sem_eval-english/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/0a8d7ea171c4abf3f72782dcec0655f511a3d8a092d4576c37d5fdd262607b2b.b1d3b19ab013240a348484166f1a44749cad6f108156f07e3784cbf6c2a27772
loading file https://huggingface.co/zdreiosis/bert-finetuned-sem_eval-english/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/zdreiosis/bert-finetuned-sem_eval-english/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/54151775412034edc448c6cf796da7959a2fb14433b5f673adf4b6019272da6b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
load

In [ ]:
labels = [label for label in raw_datasets['train'].features.keys() if label not in ['ID', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Adventure',
 'Angst',
 'Drama',
 'Family',
 'Friendship',
 'Humor',
 'Hurt_Comfort',
 'Romance']

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

In [ ]:
encoded_dataset = raw_datasets.map(preprocess_data, batched=True, remove_columns=raw_datasets['train'].column_names)
encoded_dataset.set_format("torch")

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("zdreiosis/bert-finetuned-sem_eval-english", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

loading configuration file https://huggingface.co/zdreiosis/bert-finetuned-sem_eval-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c12b5f7d968e8a2a05ad8b03b96f0c1ec169ba6badeaac0ccb707a88821bdb62.37894981151fc8aa78b1f7f90d9024fcd17c57692dbd5f7d0e4efe63cc48204b
Model config BertConfig {
  "_name_or_path": "zdreiosis/bert-finetuned-sem_eval-english",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Adventure",
    "1": "Angst",
    "2": "Drama",
    "3": "Family",
    "4": "Friendship",
    "5": "Humor",
    "6": "Hurt_Comfort",
    "7": "Romance"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Adventure": 0,
    "Angst": 1,
    "Drama": 2,
    "Family": 3,
    "Friendship": 4,
    "Humor": 5,
    "H

In [ ]:
batch_size = 32
metric_name = "f1"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"ff_analysis_2",
    evaluation_strategy = "steps", #epoch
    #save_strategy = "no", #epoch
    eval_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/zdreiosis/ff_analysis_2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.47k/418M [00:00<?, ?B/s]

Download file runs/May31_03-58-43_191603b0cab5/1653969529.9888809/events.out.tfevents.1653969529.191603b0cab5.…

Download file runs/May31_04-01-40_191603b0cab5/events.out.tfevents.1653969732.191603b0cab5.73.10:  20%|##     …

Download file runs/May31_02-55-06_191603b0cab5/1653965757.3869858/events.out.tfevents.1653965757.191603b0cab5.…

...

Download file runs/May31_04-01-40_191603b0cab5/events.out.tfevents.1653970827.191603b0cab5.73.12: 100%|#######…

Clean file runs/May31_04-01-40_191603b0cab5/events.out.tfevents.1653970827.191603b0cab5.73.12: 100%|##########…

Download file training_args.bin: 100%|##########| 2.86k/2.86k [00:00<?, ?B/s]

Clean file training_args.bin:  35%|###4      | 1.00k/2.86k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1558
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1470


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
50,No log,0.113091,0.954064,0.965127,0.931034
100,No log,0.107824,0.946619,0.958182,0.919540
150,No log,0.107388,0.957447,0.966033,0.931034
200,No log,0.103927,0.957447,0.966033,0.931034
250,No log,0.107561,0.954064,0.965127,0.931034
300,No log,0.103382,0.954064,0.965127,0.931034
350,No log,0.104577,0.946619,0.958182,0.919540
400,No log,0.103414,0.954064,0.965127,0.931034
450,No log,0.106021,0.954064,0.965127,0.931034
500,0.037700,0.108612,0.954064,0.965127,0.931034


***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
Saving model checkpoint to ff_analysis_2/checkpoint-500
Configuration saved in ff_analysis_2/checkpoint-500/config.json
Model weights saved in ff_analysis_2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ff_analysis_2/checkpoint-500/tokenizer_config.json
Special tokens file saved in ff_analys

TrainOutput(global_step=1470, training_loss=0.01877100710966149, metrics={'train_runtime': 1026.3771, 'train_samples_per_second': 45.539, 'train_steps_per_second': 1.432, 'total_flos': 3074618307502080.0, 'train_loss': 0.01877100710966149, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 87
  Batch size = 32


{'epoch': 30.0,
 'eval_accuracy': 0.9310344827586207,
 'eval_f1': 0.9540636042402827,
 'eval_loss': 0.10861221700906754,
 'eval_roc_auc': 0.9651268115942029,
 'eval_runtime': 0.6647,
 'eval_samples_per_second': 130.888,
 'eval_steps_per_second': 4.513}

In [ ]:
trainer.push_to_hub(tags="6th", commit_message="30epx/n1")

Saving model checkpoint to ff_analysis_2
Configuration saved in ff_analysis_2/config.json
Model weights saved in ff_analysis_2/pytorch_model.bin
tokenizer config file saved in ff_analysis_2/tokenizer_config.json
Special tokens file saved in ff_analysis_2/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.9540636042402827}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9310344827586207}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/zdreiosis/ff_analysis_2
   e646d9c..b2f71e3  main -> main



In [ ]:
#text = "Kagome tricks Sesshomaru into jumping down the well with her, only she wasn't expecting it to work! Now she's stuck with him as the return trip goes wonky, throwing them back to his father's time. Although, a Japan with no jewel shards certainly appeals to Kagome. Can she keep it that way? AU?"

#encoding = tokenizer(text, return_tensors="pt")
#encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

#outputs = trainer.model(**encoding)

#logits = outputs.logits
#logits.shape

## apply sigmoid + threshold
#sigmoid = torch.nn.Sigmoid()
#probs = sigmoid(logits.squeeze().cpu())
#predictions = np.zeros(probs.shape)
#predictions[np.where(probs >= 0.15)] = 1 #>= 0.5 orig. but considering the weird data...
## turn predicted id's into actual label names
#predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
#print(predicted_labels)

#this part isn't actually used

['Romance']


AAAAAAAA